Setup delle variabili generali

In [1]:
from main import *

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 150)

studyId = 'nsclc_ctdx_msk_2022'
#studyId = 'msk_impact_2017'
#studyId = 'msk_met_2021'
#studyId = 'pancan_pcawg_2020'

Importazione dei dati

In [2]:
patient_data = get_PatientData(studyId)
#display(patient_data)

sample_data = get_SampleData(studyId)
#display(sample_data)

mutation_data = get_MutationData(studyId)
#display(mutation_data)

full_data = get_FullData(sample_data, mutation_data)
#display(full_data)

Costruzione del grafo

In [3]:
dip_graph = build_DiPGraph(full_data)
pm_graph = build_PMGraph(full_data, patient_data, sample_data)

Vista sulle malattie

In [ ]:
df0 = show_diseases(dip_graph)

display(df0)
#with pd.option_context('display.max_rows', None): display(df)

Vista di una malattia specifica: pazienti e mutazioni

In [ ]:
disease = 'all'
kmin = 0
kmax = 1000
genes = set()
status = 'all'
patients = getPatients_fromDisease(dip_graph, pm_graph, disease, kmin, kmax, genes, status, no_check=False)

df1 = view_patients(pm_graph, patients)

df2 = getMutations_fromPatients(pm_graph, patients)

df3 = df2.groupby('Gene')['% Pazienti'].sum().reset_index(name='% Pazienti')
df3.sort_values(by='% Pazienti', ascending=False, inplace=True)
df3.reset_index(drop=True, inplace=True)

df4 = getGenes_fromPatients(pm_graph, patients)

print(f'Studio: {studyId}')
print(f'Malattia: {disease}')
print('')
print(f'Survival rate minimo: {kmin}')
print(f'Surival rate massimo: {kmax}')
print(f'Numero pazienti: {len(patients)}')
avg_survival_rate = round(df1['Survival rate'].mean(), 1)
print(f'Average survival rate: {avg_survival_rate}')

survival = df1.value_counts('Survival status', normalize=True, sort=False).mul(100).round(1)
print(survival)

display(df1)
print('Tabella delle percetuali di ogni mutazione')
display(df2)
print('Tabella delle percentuali di ogni gene (calcolata come somma delle percentuali delle mutazioni)')
display(df3)
print('Tabella delle percentuali di ogni gene')
display(df4)
#with pd.option_context('display.max_rows', None): display(df2)

position = f'risultati/esp 4'
#save_result(df1, position, f'{studyId} pazienti')
#save_result(df2, position, f'{studyId} mutazioni')
#save_result(df3, position, 'geni (multi-mutazioni)')
#save_result(df4, position, f'{studyId} geni')

Sottoinsiemi massimali di mutazioni

In [ ]:
def first_check(pm_graph, P, m, k):
    c = pm_graph.in_degree(m)
    if c/len(P) >= k/100:
        return True
    else:
        return False

def check(pm_graph, P, M, k):
    c = 0
    for p in P:
        muts = pm_graph.neighbors(p)
        if M.issubset(muts):
            c += 1
    if c/len(P) >= k/100:
        return True
    else:
        return False

disease = 'all'
kmin = 0
kmax = 1000
genes = set()
status = 'all'
disease = 'Lung Adenocarcinoma'
Pa = getPatients_fromDisease(dip_graph, pm_graph, disease, kmin, kmax, genes, status, no_check=True)
Ma = get_MNodes(pm_graph)
k = 10
print(f'Studio: {studyId}')
print(f'Numero pazienti: {len(Pa)}')
print(f'Numero mutazioni: {len(Ma)}')
print(f'k = {k}')

A = set()
Mu = set()
for m in Ma:
    if first_check(pm_graph, Pa, m, k):
        Mu.add(frozenset({m}))
        A.add(frozenset({m}))
S = set()
if not Mu:
    c = 0
else:
    c = 1
while c == 1:
    B = set()
    D = set()
    for ma in A:
        for mu in Mu:
            m = ma | mu
            if check(pm_graph, Pa, m, k) and m not in B:
                B.add(m)
                if ma != mu:
                    D.add(ma)
    if not B:
        c = 0
    else:
        S = S | B
        S = S - D
        if A != B:
            A = B
        else:
            c = 0

l = list(S)
df = pd.DataFrame(l)
display(df)
#save_result(df, studyId, f'{k}%')

Clustering

In [4]:
k = 0
#disease = 'Lung Adenocarcinoma'
#patients = list(p for p in dip_graph.neighbors(disease))
patients = get_PNodes(pm_graph)
total_patients = len(patients)
clusters = clustering(pm_graph, patients, k)

In [ ]:
if len(clusters) > 0:
    cluster_view = {'Cluster' : [], '# Pazienti' : [], '# Mutazioni': []}
    for n, patients in clusters.items():
        cluster_view['Cluster'].append(n)
        cluster_view['# Pazienti'].append(len(patients))
        mutations_df = getMutations_fromPatients(pm_graph, patients)
        cluster_view['# Mutazioni'].append(len(mutations_df.index))
    cluster_view_df = pd.DataFrame(cluster_view)

    cluster_patients = cluster_view_df['# Pazienti'].sum()
    ones = total_patients - cluster_patients
    total_clusters = len(clusters) + ones
    studyName = 'MSK-IMPACT Clinical Sequencing Cohort (MSK, Nat Med 2017)'
    cluster_view_df.iloc[:].plot.bar(x='Cluster', rot=0, title=f'Studio: {studyName} - {total_patients} pazienti - k = {k}', figsize=(20, 5))
    with pd.option_context('display.max_rows', 50):
        display(cluster_view_df)
    print(f'{total_patients} pazienti\n{total_clusters} cluster totali: {len(clusters)} cluster con piú di un paziente ({cluster_patients} pazienti), {ones} cluster con un paziente')

    position = f'risultati/clustering/jaccard'
    #save_result(cluster_view_df, position, f'{studyId} (k = {k})')

In [ ]:
def show_cluster(number):
    for n, patients in clusters.items():
        if n == number or number == -1:
            patients_df = view_patients(pm_graph, patients)
            mutations_df = getMutations_fromPatients(pm_graph, patients)
            print(f'Cluster {n}')
            display(patients_df)
            display(mutations_df)
            #save_result(patients_df, position, f'{studyId} (k = {k}) - cl{number} pazienti')
            #save_result(mutations_df, position, f'{studyId} (k = {k}) - cl{number} mutazioni')

show_cluster(-1)